Buenos días equipo *Public Affairs*,

Este es mi proyecto de automatización de la tarea de monitorización legal diaria. Iré explicando los pasos seguidos para que sea comprensible/accesible para todos.

In [ ]:
!pip install requests beautifulsoup4 selenium pdfminer.six pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 25.1 MB/s eta 0:00:00


***BOLETÍN OFICIAL DEL ESTADO***

1. Descarga automática del Boletín.
2. Definir función para buscar en el BOE.
3. Ejecutar función y devolver disposiciones más urls.



In [ ]:
from google.colab import drive
from pdfminer.high_level import extract_text
from pdfminer.pdfpage import PDFPage
import requests
from datetime import datetime, timedelta
import re

# Paso 1: Montar Google Drive
drive.mount('/content/drive')

# Función para calcular el número del boletín excluyendo domingos
def calcular_numero_boletin(fecha_base, numero_base, fecha_actual):
    """
    Calcula el número del boletín excluyendo domingos.
    """
    dias_habiles = 0
    fecha = fecha_base

    while fecha <= fecha_actual:
        if fecha.weekday() != 6:  # Excluir domingos (weekday == 6)
            dias_habiles += 1
        fecha += timedelta(days=1)

    return numero_base + dias_habiles - 1

# Función para generar la URL del BOE
def generar_url_boe(fecha_actual, numero_boletin):
    """
    Genera la URL del BOE según la fecha actual y el número de boletín.
    """
    año = fecha_actual.strftime("%Y")
    mes = fecha_actual.strftime("%m")
    dia = fecha_actual.strftime("%d")
    return f"https://www.boe.es/boe/dias/{año}/{mes}/{dia}/pdfs/BOE-S-{año}-{numero_boletin}.pdf"

# Función para descargar el BOE
def descargar_boe(url, nombre_archivo):
    """
    Descarga el BOE desde la URL especificada.
    """
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(nombre_archivo, 'wb') as file:
                file.write(response.content)
            print(f"Archivo descargado y guardado como: {nombre_archivo}")
            return True
        else:
            print(f"No se pudo descargar el archivo. Código de respuesta: {response.status_code}")
            return False
    except Exception as e:
        print(f"Error al descargar el archivo: {e}")
        return False

# Función para buscar palabras clave en el BOE y generar enlaces
def buscar_boe(pdf_path, keywords, enlace_base):
    """
    Busca palabras clave en un PDF y devuelve los párrafos relevantes junto con enlaces generados.
    """
    try:
        with open(pdf_path, "rb") as file:
            # Procesar página por página
            for i, page in enumerate(PDFPage.get_pages(file, caching=True, check_extractable=True), start=1):
                # Extraer texto de la página actual
                page_text = extract_text(pdf_path, page_numbers=[i - 1])

                # Buscar todos los indicadores en la página
                indicadores = re.findall(r"BOE-[A-Z]-\d{4}-\d{5}", page_text)

                # Dividir el texto de la página en párrafos
                paragraphs = page_text.split("\n\n")

                for paragraph in paragraphs:
                    paragraph_clean = paragraph.strip()

                    # Comprobar si alguna palabra clave está en el párrafo
                    if any(keyword.lower() in paragraph_clean.lower() for keyword in keywords):
                        print(f"\nKeyword encontrada en la página {i}:")
                        print(f"Párrafo:\n{paragraph_clean}\n")

                        # Buscar el indicador más cercano al párrafo
                        indicador = None
                        for ind in indicadores:
                            if ind in paragraph_clean:
                                indicador = ind
                                break

                        if not indicador:
                            # Si no está en el párrafo, buscar el indicador más cercano en el texto completo
                            paragraph_position = page_text.find(paragraph_clean)
                            for ind in indicadores:
                                indicator_position = page_text.find(ind)
                                if 0 <= indicator_position - paragraph_position < 300:  # Por ejemplo, a 300 caracteres
                                    indicador = ind
                                    break

                        if indicador:
                            link_disposicion = f"{enlace_base}{indicador}.pdf"
                            print(f"Enlace generado: {link_disposicion}\n")
                        else:
                            print("No se encontró un indicador cercano a este párrafo.\n")
    except Exception as e:
        print(f"Error al procesar el PDF: {e}")

# Configuración inicial
fecha_base = datetime(2024, 11, 20)  # Fecha base conocida (boletín 280)
numero_base = 280  # Número de boletín en esa fecha base
fecha_actual = datetime.today()  # Fecha actual

# Verificar si es domingo
if fecha_actual.weekday() == 6:  # 6 es domingo
    print("Hoy es domingo, no se publica el BOE.")
else:
    # Calcular el número del boletín actual
    numero_boletin_actual = calcular_numero_boletin(fecha_base, numero_base, fecha_actual)

    # Generar la URL del BOE diario
    url_boe = generar_url_boe(fecha_actual, numero_boletin_actual)
    print(f"URL generada para el BOE diario: {url_boe}")

    # Ruta donde se guardará el PDF
    pdf_path = f"/content/drive/My Drive/BOE-{fecha_actual.strftime('%Y-%m-%d')}.pdf"

    # Descargar el BOE
    if descargar_boe(url_boe, pdf_path):
        # Paso final: Ejecutar la búsqueda en el PDF descargado
        enlace_base = f"https://www.boe.es/boe/dias/{fecha_actual.strftime('%Y/%m/%d')}/pdfs/"
        keywords = ["acceso", "Convenio"]
        buscar_boe(pdf_path, keywords, enlace_base)


Mounted at /content/drive
URL generada para el BOE diario: https://www.boe.es/boe/dias/2024/11/27/pdfs/BOE-S-2024-286.pdf
Archivo descargado y guardado como: /content/drive/My Drive/BOE-2024-11-27.pdf

Keyword encontrada en la página 4:
Párrafo:
Orden  de  11  de  noviembre  de  2024,  del  Departamento  de  Gobernanza, 
Administración Digital y Autogobierno, por la que se convoca proceso selectivo para 
el  acceso,  por  promoción  interna,  a  la  Subescala  de  Secretaría,  categoría  de 
Entrada,  de  la  Escala  de  personal  funcionario  de  administración  local  con 
habilitación  de  carácter  nacional,  en  el  ámbito  de  la  Comunidad  Autónoma  de 
Euskadi.

No se encontró un indicador cercano a este párrafo.


Keyword encontrada en la página 4:
Párrafo:
Orden  de  11  de  noviembre  de  2024,  del  Departamento  de  Gobernanza, 
Administración Digital y Autogobierno, por la que se convoca proceso selectivo para 
el acceso, por promoción interna, a la Subescala de Secretar

Lectura diaria del BOE extrayendo los párrafos y urls dónde se encuentran las keywords.

***COMUNIDAD DE MADRID***


1.   Descarga automática del Boletín.
2.   Definir función para buscar en el BOE.
3. Ejecutar función y devolver disposiciones más urls.



In [ ]:
from datetime import datetime, timedelta
import requests
from pdfminer.high_level import extract_text
from pdfminer.pdfpage import PDFPage
import re

# Función para calcular el número del boletín
def calcular_numero_boletin(fecha_base, numero_base, fecha_actual):
    """
    Calcula el número del boletín excluyendo domingos.
    Args:
        fecha_base (datetime): Fecha base conocida.
        numero_base (int): Número del boletín en la fecha base.
        fecha_actual (datetime): Fecha actual.
    Returns:
        int: Número calculado del boletín.
    """
    dias_habiles = 0
    fecha = fecha_base

    while fecha <= fecha_actual:
        if fecha.weekday() != 6:  # Excluir domingos (6)
            dias_habiles += 1
        fecha += timedelta(days=1)

    return numero_base + dias_habiles - 1

# Función para descargar PDFs del BOCM
def descargar_bocm(fecha, numero_boletin, ruta_guardado):
    if fecha.weekday() == 6:  # Verificar si es domingo
        print("El domingo no se publica el BOCM.")
        return False

    año = fecha.strftime("%Y")
    mes = fecha.strftime("%m")
    dia = fecha.strftime("%d")
    numero_boletin_completo = f"BOCM-{año}{mes}{dia}{numero_boletin}"
    url = f"https://www.bocm.es/boletin/CM_Boletin_BOCM/{año}/{mes}/{dia}/{numero_boletin_completo}.PDF"

    print(f"Descargando boletín desde: {url}")
    response = requests.get(url)
    if response.status_code == 200:
        with open(ruta_guardado, "wb") as file:
            file.write(response.content)
        print(f"Boletín descargado correctamente en: {ruta_guardado}")
        return True
    else:
        print(f"No se pudo descargar el boletín. Código de respuesta: {response.status_code}")
        return False

# Función para buscar disposiciones relevantes
def buscar_disposiciones_bocm(pdf_path, enlace_base, keyword):
    """
    Busca disposiciones en el boletín del BOCM con indicadores y genera enlaces específicos.

    Args:
        pdf_path (str): Ruta del archivo PDF.
        enlace_base (str): URL base para generar los enlaces.
        keyword (str): Palabra clave para filtrar disposiciones.

    Returns:
        None. Imprime disposiciones relevantes y sus enlaces.
    """
    try:
        # Abrir el PDF y procesar cada página individualmente
        with open(pdf_path, "rb") as file:
            for i, page in enumerate(PDFPage.get_pages(file, caching=True, check_extractable=True), start=1):
                # Extraer texto de la página actual
                page_text = extract_text(pdf_path, page_numbers=[i - 1])

                # Buscar todos los indicadores en la página
                indicadores = re.findall(r"BOCM-\d{8}-\d+", page_text)

                # Dividir el texto en párrafos
                parrafos = page_text.split("\n\n")
                for parrafo in parrafos:
                    parrafo_limpio = parrafo.strip().replace("\n", " ")  # Normalizar texto
                    if keyword.lower() in parrafo_limpio.lower():
                        # Buscar el indicador más cercano al párrafo
                        indicador = None
                        for ind in indicadores:
                            if ind in page_text:
                                indicador = ind
                                break

                        if indicador:
                            enlace = f"{enlace_base}{indicador}.PDF"
                            print(f"\nDisposición encontrada en la página {i}:")
                            print(f"Texto completo:\n{parrafo.strip()}\n")
                            print(f"Enlace generado: {enlace}\n")
                        else:
                            print(f"\nNo se encontró un indicador único para este párrafo en la página {i}.")
    except Exception as e:
        print(f"Error al procesar el PDF: {e}")

# Configuración inicial
fecha_base = datetime(2024, 11, 18)  # Fecha base conocida (boletín 275)
numero_base = 275
fecha_actual = datetime.today()  # Fecha actual

# Calcular el número del boletín actual
if fecha_actual.weekday() == 6:  # Verificar si es domingo
    print("Hoy es domingo, no se publica el BOCM.")
else:
    numero_boletin_actual = calcular_numero_boletin(fecha_base, numero_base, fecha_actual)

    # Ruta para guardar el boletín actual
    pdf_path_actual = "/content/BOCM_boletin_actual.pdf"

    # Descargar el boletín
    if descargar_bocm(fecha_actual, numero_boletin_actual, pdf_path_actual):
        # URL base para los enlaces del boletín actual
        enlace_base_actual = f"https://www.bocm.es/boletin/CM_Orden_BOCM/{fecha_actual.strftime('%Y/%m/%d')}/"

        # Palabra clave que deseas buscar
        keyword = "mental"  # Cambia esta palabra clave según lo que necesites

        # Probar la función con el boletín actual
        buscar_disposiciones_bocm(pdf_path_actual, enlace_base_actual, keyword)


Descargando boletín desde: https://www.bocm.es/boletin/CM_Boletin_BOCM/2024/11/26/BOCM-20241126282.PDF
Boletín descargado correctamente en: /content/BOCM_boletin_actual.pdf


***COMUNIDAD VALENCIANA***


1.   Descarga automática del Boletín.
2. Definir función para buscar en el BOE.
3. Ejecutar función y devolver disposiciones más urls.



In [ ]:
from datetime import datetime, timedelta
import requests
from pdfminer.high_level import extract_text
from pdfminer.pdfpage import PDFPage
import re

# Función para calcular el número del boletín
def calcular_numero_boletin(fecha_base, numero_base, fecha_actual):
    """
    Calcula el número del boletín basado en los días hábiles desde la fecha base.

    Args:
        fecha_base (datetime): Fecha base conocida para la numeración.
        numero_base (int): Número del boletín en la fecha base.
        fecha_actual (datetime): Fecha actual para la cual calcular el número.

    Returns:
        int: Número calculado del boletín.
    """
    dias_habiles = 0
    fecha = fecha_base

    while fecha <= fecha_actual:
        if fecha.weekday() != 5:  # Ignorar sábados
            dias_habiles += 1
        fecha += timedelta(days=1)

    return numero_base + dias_habiles - 1

# Función para descargar PDFs del DOGV
def descargar_dogv(fecha, numero_boletin, ruta_guardado):
    if fecha.weekday() == 5:  # Verificar si es sábado
        print("El sábado no se publica el DOGV.")
        return False

    año = fecha.strftime("%Y")
    mes = fecha.strftime("%m")
    dia = fecha.strftime("%d")
    numero_boletin_completo = f"sumario_{año}_{numero_boletin}_es"
    url = f"https://dogv.gva.es/datos/{año}/{mes}/{dia}/pdf/{numero_boletin_completo}.pdf"

    print(f"Intentando descargar boletín desde: {url}")
    response = requests.get(url)
    if response.status_code == 200:
        with open(ruta_guardado, "wb") as file:
            file.write(response.content)
        print(f"Boletín descargado correctamente en: {ruta_guardado}")
        return True
    else:
        print(f"No se pudo descargar el boletín. Código de respuesta: {response.status_code}")
        return False

# Función para buscar disposiciones relevantes
def buscar_disposiciones_dogv(pdf_path, enlace_base, keyword):
    """
    Busca disposiciones en el boletín del DOGV con indicadores y genera enlaces específicos.

    Args:
        pdf_path (str): Ruta del archivo PDF.
        enlace_base (str): URL base para generar los enlaces.
        keyword (str): Palabra clave para filtrar disposiciones.

    Returns:
        None. Imprime disposiciones relevantes y sus enlaces.
    """
    try:
        # Abrir el PDF y procesar cada página individualmente
        with open(pdf_path, "rb") as file:
            for i, page in enumerate(PDFPage.get_pages(file, caching=True, check_extractable=True), start=1):
                # Extraer texto de la página actual
                page_text = extract_text(pdf_path, page_numbers=[i - 1])

                # Buscar todos los indicadores en la página
                indicadores = re.findall(r"DOGV-C-\d{4}-\d+", page_text)

                # Dividir el texto en párrafos
                parrafos = page_text.split("\n\n")
                for parrafo in parrafos:
                    parrafo_limpio = parrafo.strip().replace("\n", " ")  # Normalizar texto
                    if keyword.lower() in parrafo_limpio.lower():
                        # Buscar el indicador más cercano al párrafo
                        indicador = None
                        for ind in indicadores:
                            if ind in page_text:
                                indicador = ind


                        if indicador:
                            # Extraer los últimos 5 dígitos del indicador y construir el enlace
                            ultimos_cinco_digitos = indicador.split("-")[-1]
                            año = datetime.today().strftime("%Y")  # Año actual
                            enlace = f"{enlace_base}{año}_{ultimos_cinco_digitos}_es.pdf"
                            print(f"\nDisposición encontrada en la página {i}:")
                            print(f"Texto completo:\n{parrafo.strip()}\n")
                            print(f"Enlace generado: {enlace}\n")
                        else:
                            print(f"\nNo se encontró un indicador único para este párrafo en la página {i}.")
    except Exception as e:
        print(f"Error al procesar el PDF: {e}")

# Configuración para el boletín de hoy
fecha_base = datetime(2024, 11, 18)  # Fecha base conocida (lunes con boletín 9986)
numero_base = 9986
fecha_hoy = datetime.today()  # Fecha de hoy
numero_boletin_hoy = calcular_numero_boletin(fecha_base, numero_base, fecha_hoy)

# Ruta para guardar el boletín de hoy
pdf_path_hoy = "/content/DOGV_boletin_hoy.pdf"

# Descargar el boletín de hoy
if descargar_dogv(fecha_hoy, numero_boletin_hoy, pdf_path_hoy):
    # URL base para los enlaces del boletín de hoy
    enlace_base_hoy = f"https://dogv.gva.es/datos/{fecha_hoy.strftime('%Y/%m/%d')}/pdf/"

    # Palabra clave que deseas buscar
    keyword = "mujeres"  # Cambia esta palabra clave según lo que necesites

    # Probar la función con el boletín de hoy
    buscar_disposiciones_dogv(pdf_path_hoy, enlace_base_hoy, keyword)


Intentando descargar boletín desde: https://dogv.gva.es/datos/2024/11/29/pdf/sumario_2024_9996_es.pdf
Boletín descargado correctamente en: /content/DOGV_boletin_hoy.pdf


***PAÍS VASCO***
1. Descarga automática del Boletín.
2. Definir función para buscar en el Boletín del País Vasco.
3. Ejecutar función y devolver disposiciones más urls.


BOAM, no te devuelve link, pero bueno algo servirá

In [ ]:
from google.colab import drive
import requests
from pdfminer.high_level import extract_text
from pdfminer.pdfpage import PDFPage

# Montar Google Drive
drive.mount('/content/drive')

# Función para descargar el PDF y guardarlo en Google Drive
def descargar_pdf_a_drive(url, ruta_drive):
    """
    Descarga el PDF desde la URL especificada y lo guarda en Google Drive.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            with open(ruta_drive, 'wb') as file:
                file.write(response.content)
            print(f"Archivo descargado y guardado en Google Drive: {ruta_drive}")
            return True
        else:
            print(f"No se pudo descargar el archivo. Código de respuesta: {response.status_code}")
            return False
    except Exception as e:
        print(f"Error al descargar el archivo: {e}")
        return False

# Función para buscar palabras clave en el PDF
def buscar_palabras_pdf(pdf_path, keywords):
    """
    Busca palabras clave en un PDF y devuelve los párrafos relevantes con la página en la que se encuentran.

    Args:
        pdf_path (str): Ruta al archivo PDF.
        keywords (list): Lista de palabras clave a buscar.

    Returns:
        None. Imprime los resultados directamente.
    """
    try:
        with open(pdf_path, "rb") as file:
            # Procesar página por página
            for i, page in enumerate(PDFPage.get_pages(file, caching=True, check_extractable=True), start=1):
                # Extraer texto de la página actual
                page_text = extract_text(pdf_path, page_numbers=[i - 1])

                # Dividir el texto de la página en párrafos
                paragraphs = page_text.split("\n\n")

                for paragraph in paragraphs:
                    paragraph_clean = paragraph.strip()

                    # Comprobar si alguna palabra clave está en el párrafo
                    if any(keyword.lower() in paragraph_clean.lower() for keyword in keywords):
                        print(f"\nKeyword encontrada en la página {i}:")
                        print(f"Párrafo:\n{paragraph_clean}\n")
    except Exception as e:
        print(f"Error al procesar el PDF: {e}")

# Configuración inicial
if __name__ == "__main__":
    # URL del PDF específico
    url_pdf = "https://www.asambleamadrid.es/static/doc/publicaciones/BOAM_13_00082.pdf"

    # Ruta en Google Drive donde se guardará el PDF
    ruta_drive_pdf = "/content/drive/My Drive/BOAM_13_00082.pdf"

    # Lista de palabras clave a buscar
    keywords = ["acceso", "Convenio"]  # Cambia las palabras clave según tus necesidades

    # Descargar el PDF y guardarlo en Google Drive
    if descargar_pdf_a_drive(url_pdf, ruta_drive_pdf):
        # Ejecutar la búsqueda en el PDF descargado
        buscar_palabras_pdf(ruta_drive_pdf, keywords)
